In [35]:
import json
import pandas as pd
from tqdm.notebook import tqdm
import numpy as np
import os

data = []
with open("data.json", 'r') as f:
    for line in f:
        data.append(json.loads(line))

In [36]:
dict_ = {'id':[], 'submitter':[], 'abstract': [], 'authors': [], 'title': [], 'references': [], 'license': [], 'doi': []}
for article in data:
    dict_['id'].append(article['id'])
    dict_['submitter'].append(article['submitter'])
    dict_['title'].append(article['title'])
    dict_['authors'].append(article['authors'])
    dict_['abstract'].append(article['abstract'])
    dict_['references'].append(article['journal-ref'])
    dict_['license'].append(article['license'])
    dict_['doi'].append(article['doi'])
df = pd.DataFrame(dict_, columns = ['id', 'submitter', 'title', 'authors', 'abstract', 'references', 'license', 'doi'])
df.head(5)

,id,submitter,title,authors,abstract,references,license,doi
0,0704.0001,Pavel Nadolsky,Calculation of prompt diphoton production cros...,"C. Bal\'azs, E. L. Berger, P. M. Nadolsky, C.-...",A fully differential calculation in perturba...,"Phys.Rev.D76:013009,2007",None,10.1103/PhysRevD.76.013009
1,0704.0002,Louis Theran,Sparsity-certifying Graph Decompositions,Ileana Streinu and Louis Theran,"We describe a new algorithm, the $(k,\ell)$-...",None,http://arxiv.org/licenses/nonexclusive-distrib...,None
2,0704.0003,Hongjun Pan,The evolution of the Earth-Moon system based o...,Hongjun Pan,The evolution of Earth-Moon system is descri...,None,None,None
3,0704.0004,David Callan,A determinant of Stirling cycle numbers counts...,David Callan,We show that a determinant of Stirling cycle...,None,None,None
4,0704.0005,Alberto Torchinsky,From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...,Wael Abu-Shammala and Alberto Torchinsky,In this paper we show how to compute the $\L...,"Illinois J. Math. 52 (2008) no.2, 681-689",None,None


In [37]:
newFeatures = []
for article in tqdm(data):
    features = [article['id'],
                article['title'],
               article['submitter'],
               article['authors'],
               article['abstract'],
               article['journal-ref'],
               article['license'],
               article['doi']]
    newFeatures.append(features)
    
    

  0%|          | 0/1796911 [00:00<?, ?it/s]

In [38]:
col = ['Paper ID',
      'Title',
      'Submitter',
      'Authors',
      'Abstract',
       'References',
      'License',
      'Doi']
newDF = pd.DataFrame(newFeatures, columns = col)
newDF.head()

,Paper ID,Title,Submitter,Authors,Abstract,References,License,Doi
0,0704.0001,Calculation of prompt diphoton production cros...,Pavel Nadolsky,"C. Bal\'azs, E. L. Berger, P. M. Nadolsky, C.-...",A fully differential calculation in perturba...,"Phys.Rev.D76:013009,2007",None,10.1103/PhysRevD.76.013009
1,0704.0002,Sparsity-certifying Graph Decompositions,Louis Theran,Ileana Streinu and Louis Theran,"We describe a new algorithm, the $(k,\ell)$-...",None,http://arxiv.org/licenses/nonexclusive-distrib...,None
2,0704.0003,The evolution of the Earth-Moon system based o...,Hongjun Pan,Hongjun Pan,The evolution of Earth-Moon system is descri...,None,None,None
3,0704.0004,A determinant of Stirling cycle numbers counts...,David Callan,David Callan,We show that a determinant of Stirling cycle...,None,None,None
4,0704.0005,From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...,Alberto Torchinsky,Wael Abu-Shammala and Alberto Torchinsky,In this paper we show how to compute the $\L...,"Illinois J. Math. 52 (2008) no.2, 681-689",None,None


In [39]:
newDF.to_csv('data.csv', index = False)


In [40]:
from whoosh.index import create_in
from whoosh.fields import *
from whoosh.qparser import QueryParser
from whoosh.query import *
from whoosh.index import open_dir
from whoosh.analysis import StandardAnalyzer
from whoosh.analysis import Tokenizer, Token

In [41]:
readDF = pd.read_csv('data.csv')
readDF.shape

    

/Users/karthik1391/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


(1796911, 8)

In [42]:
readDF.columns

Index(['Paper ID', 'Title', 'Submitter', 'Authors', 'Abstract', 'References',
       'License', 'Doi'],
      dtype='object')

In [63]:
schema = Schema(id = ID(stored = True), submitter = TEXT(stored = True), 
                authors = TEXT(stored = True), abstract = TEXT(stored = True), title = TEXT(stored = True),
                references = TEXT(stored = True), doi = ID(stored = True), license = ID(stored = True))
ix = create_in(".", schema)
print(ix)

FileIndex(FileStorage('.'), 'MAIN')


In [84]:
writer = ix.writer()
for index, row in readDF.iterrows():
    writer.add_document(authors = row.get("Authors"), title = row.get("Title"), abstract = row.get("Abstract"))
    #writer.add_document(id = row.get("id"), abstract = row.get("abstract"))
    #writer.add_document(ids = row.get("id"), title = row.get("title"), authors = row.get("authors"), abstract = row.get("abstract"), 
    #submitter = row.get("submitter"), doi = row.get("doi"), references = row.get("references"), license = row.get("license"))
writer.commit()

        

In [74]:
writer = ix.writer()
writer.add_document(title = u"My document", abstract = u"This is a test")
writer.add_document(title = u"My doc 2", abstract = u"Best song ever is Eye of the Tiger")
writer.add_document(title = u"Tupac Shakur", abstract = u"Dear Mama is this artist's best ever song")
writer.add_document(title = u"Amma's favorite song", abstract = u"Favorite song is Best Day of My Life")
writer.commit()

In [87]:
searcher = ix.searcher()
proposed_time_strings = []
with ix.searcher() as searcher:
    parser = QueryParser("title", ix.schema)
    querystring = u"airplane"
    myquery = parser.parse(querystring)
    print(myquery)
    results = searcher.search(myquery)
    results.fragmenter.surround = 500
    results.fragmenter.maxchars = 1500
    print(results)
    print()
    for res in results:
        print(res["title"])
        print(res["abstract"])
#work on the search string everything else seems fine


title:airplane
<Top 10 Results for Term('title', 'airplane') runtime=0.0020035440029460005>

The Story of Airplane Wings
  The purpose of this work is to explain how wings work and how they were
invented. We use the lens of history, looking at the individual people who
wanted to fly, the lens of technology, looking at the key inventions leading up
to modern airplanes, and the lens of physics, looking at the equations of
airflow that made it all possible. Finally, we derive our own wing from
scratch. We do this by simulating a wind tunnel, placing a rectangular
occlusion in it, and then using gradient ascent to turn it into a wing.

Experimental test of airplane boarding methods
  We report the results of an experimental comparison of different airplane
boarding methods. This test was conducted in a mock 757 fuselage, located on a
Southern California soundstage, with 12 rows of six seats and a single aisle.
Five methods were tested using 72 passengers of various ages. We found a
signifi